**PDF READER** 

Il seguente pdf reader permette di convertire il contenuto presente in documenti pdf da url e trasformarlo in testo per poter effettuare operazioni di scraping o altro. Le url possono fare riferimento sia a pdf scannerizzati che puramente testuali: l'implementazione permette infatti di distinguere tra pdf scannerizzati e non scannerizzati ed adattare il modello più adatto ottimizzando tempi ed esecuzione.

---
Made by DESTRO GABRIELE, with love

In [1]:
!apt-get install poppler-utils -q
!pip install pymupdf -q
!pip install easyocr -q
!pip install pdf2image -q
!pip install transformers -q

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 1s (134 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 128208 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.86.1-0ubuntu1.1_amd64.deb ...
Unpacking poppler-utils (0.86.1-0ubuntu1.1) ...
Setting up poppler-utils (0.86.1-0ubuntu1.1) ...
Processing triggers for man-db (2.9.1-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.

In [2]:
import io
import requests
import fitz
import easyocr
import pdf2image
import pandas as pd
from transformers import pipeline

In [3]:
def initialize_reader():
  reader = easyocr.Reader(['it'], gpu = True)
  return reader

In [4]:
def initialize_transformer(model):
  transformer = pipeline('question-answering', model = model)
  return transformer

In [5]:
def get_content(url):
  try:
    response = requests.get(url)
    req_content = response.content
  except:
    print('Request failed')
  return req_content

In [20]:
def pdf_read(pdf):
  try:
   with fitz.open(stream=pdf) as doc:
     text = ""
     for idx, page in enumerate(doc):
        text += page.get_text()
  except:
     text = 'Broken'
  return text

In [21]:
def pdf_ocr(scanned_pdf, reader):
  try:
    img = pdf2image.convert_from_bytes(scanned_pdf, fmt = 'jpeg')
    txt = ""
    for page in img:
      text = reader.readtext(page, batch_size = 50, detail = 0)
      final = " ".join(text)
      txt += final + "\n"
  except:
    txt = 'Broken'
  return txt

In [8]:
def pdf_text_retriever(document_url, ocr):
  try:
    content = get_content(document_url)
    extracted_txt = pdf_read(content)
    if len(extracted_txt) < 100:
      extracted_txt = pdf_ocr(content, ocr)
  except:
    extracted_txt = 'Broken'
  return extracted_txt

In [9]:
def question_answering(context, question, transformer):
  answer = transformer(context=context, question=question)
  return(answer)

In [10]:
def get_answers(documents_urls, question_to_ask, model_id):
  r_ocr = initialize_reader()
  transformer = initialize_transformer(model = model_id)
  answers = []
  for idx, url in enumerate(documents_urls):
    if (url[-4:]!='.pdf'):
      print(f'Url n. {idx+1} does not contain a PDF')
      answers.append('Broken')
      continue
    answers.append((question_answering(pdf_text_retriever(url, r_ocr), question_to_ask, transformer)))
    print(f'Pdf n. {idx+1} processed successfully')
  dt = {'Urls': documents_urls,
        'Answers': answers}
  data = pd.DataFrame(dt)
  print('Dataframe created successfully:')
  return data

  if type(documents_urls) != list:
    temp = documents_urls
    documents_urls = []
    documents_urls.append(temp)

PROVIAMO IL PDF READER

In [11]:
URL_read = 'https://pvp.giustizia.it/pvp-resources/cms/documents/2017/20170831/b8a2599397896f067cf27f551865f914.pdf'
URL_ocr = 'https://pvp.giustizia.it/pvp-resources/cms/documents/2017/20170801/fb22c917cd6887bedd22308e0ce50d93.pdf'
URL_img = 'https://pvp.giustizia.it/pvp-resources/cms/documents/2020/20200204/df75a3eb43a87a3fff651daf257966cb.jpg'
URL_test = 'https://pvp.giustizia.it/pvp-resources/cms/documents/2017/20170913/396d564d3960ab60743776149f552fd2.pdf'

In [12]:
prova = []
prova.append(URL_read)
prova.append(URL_ocr)

In [26]:
question = "Qual'è la dimensione della casa in mq?"
m = 'antoniocappiello/bert-base-italian-uncased-squad-it'

In [27]:
a = get_answers(documents_urls = prova, question_to_ask = question, model_id = m)
a

Pdf n. 1 processed successfully
Pdf n. 2 processed successfully
Dataframe created successfully:


,Urls,Answers
0,https://pvp.giustizia.it/pvp-resources/cms/doc...,"{'score': 0.9049052596092224, 'start': 10878, ..."
1,https://pvp.giustizia.it/pvp-resources/cms/doc...,"{'score': 0.5969374179840088, 'start': 1492, '..."
